In [8]:
from sagemaker import get_execution_role

role = get_execution_role()
bucket = 'companyforu'
import pandas as pd

In [9]:
%%time
import pickle, gzip, numpy, urllib.request, json

# Load the dataset
urllib.request.urlretrieve("https://s3-us-west-1.amazonaws.com/companyforu/reviews.csv", "reviews.csv")
with gzip.open('reviews.csv', 'rb') as f:
    train_set, valid_set, test_set = pickle.load(f, encoding='latin1')

CPU times: user 841 ms, sys: 331 ms, total: 1.17 s
Wall time: 2.36 s

In [178]:
company_overall_rating = raw[['company', 'overall-ratings']].groupby(['company'], as_index = False)['overall-ratings'].mean()

In [179]:
company_overall_rating

,company,overall-ratings
0,amazon,3.587363
1,apple,3.958224
2,facebook,4.511950
3,google,4.339430
4,microsoft,3.816564
5,netflix,3.411111


In [180]:
raw['company'].value_counts()

amazon       26430
microsoft    17930
apple        12950
google        7819
facebook      1590
netflix        810
Name: company, dtype: int64

In [4]:
raw['company'].unique().tolist()

['google', 'amazon', 'facebook', 'netflix', 'apple', 'microsoft']

In [5]:
raw['work-balance-stars'].unique().tolist()

['4.0', '2.0', '5.0', 'none', '3.0', '1.0', '4.5', '2.5', '3.5', '1.5']

In [6]:
raw['culture-values-stars'].unique().tolist()

['5.0', '3.0', '4.0', 'none', '2.0', '1.0']

In [7]:
raw['carrer-opportunities-stars'].unique().tolist()

['5.0', '3.0', '4.0', 'none', '2.0', '1.0', '3.5', '2.5', '4.5', '1.5']

In [8]:
raw['comp-benefit-stars'].unique().tolist()

['4.0', '5.0', 'none', '3.0', '2.0', '1.0', '4.5', '3.5', '2.5', '1.5']

In [9]:
raw['senior-mangemnet-stars'].unique().tolist()

['5.0', '3.0', '4.0', 'none', '1.0', '2.0', '4.5', '3.5', '2.5', '1.5']

In [10]:
raw.shape

(67529, 16)

In [13]:
len(raw['location'].unique().tolist())

2044

In [14]:
location_none = raw.loc[raw['location'] == 'none']

In [15]:
location_none.shape

(25085, 16)

In [16]:
model_data = raw.loc[raw['work-balance-stars'] != 'none']
model_data = model_data.loc[raw['culture-values-stars'] != 'none']
model_data = model_data.loc[raw['carrer-opportunities-stars'] != 'none']
model_data = model_data.loc[raw['comp-benefit-stars'] != 'none']
model_data = model_data.loc[raw['senior-mangemnet-stars'] != 'none']

In [17]:
model_data.shape

(53222, 16)

In [21]:
model_data1.dtypes

company                       object
work-balance-stars            object
culture-values-stars          object
carrer-opportunities-stars    object
comp-benefit-stars            object
senior-mangemnet-stars        object
dtype: object

In [22]:
model = model_data1.copy()

In [23]:
model.loc[:, 'work-balance-stars'] = model_data1['work-balance-stars'].astype(float).round()
model.loc[:, 'culture-values-stars'] = model_data1['culture-values-stars'].astype(float).round()
model.loc[:, 'carrer-opportunities-stars'] = model_data1['carrer-opportunities-stars'].astype(float).round()
model.loc[:, 'comp-benefit-stars'] = model_data1['comp-benefit-stars'].astype(float).round()
model.loc[:, 'senior-mangemnet-stars'] = model_data1['senior-mangemnet-stars'].astype(float).round()

In [24]:
model.dtypes

company                        object
work-balance-stars            float64
culture-values-stars          float64
carrer-opportunities-stars    float64
comp-benefit-stars            float64
senior-mangemnet-stars        float64
dtype: object

In [88]:
model1 = model.drop(['company'], axis=1)

In [89]:
label = model[['company']]

In [91]:
x = model1.iloc[:, :].values 
y = label.iloc[:, :].values

In [93]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20) 

In [3]:
# logistic regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(x_train,y_train)
y_pred=logreg.predict(x_test)

In [6]:
from sagemaker import KNN

data_location = 's3://{}/companyforu/data'.format(bucket)
output_location = 's3://{}/companyforu/output'.format(bucket)

print('training data will be uploaded to: {}'.format(data_location))
print('training artifacts will be uploaded to: {}'.format(output_location))

knn = KNN(role=role,
                train_instance_count=2,
                train_instance_type='ml.x4.large',
                output_path=output_location,
                k=5,
                data_location=data_location)

training data will be uploaded to: s3://companyforu/data

training artifacts will be uploaded to: s3://companyforu/output

In [7]:
knn.fit(knn.record_set(x_train,y_train))

INFO:sagemaker:Creating training-job with name: kmeans-2019-05-01-02-24-41-409


In [10]:
kmeans_predictor = kmeans.deploy(initial_instance_count=1,
                                 instance_type='ml.x4.large')

INFO:sagemaker:Deployment finished with name: kmeans-2019-05-01-02-31-41-409